In [1]:
import pandas as pd
import numpy as np
import scanpy
import scipy
from sklearn.decomposition import PCA
from functools import partial
# from scipy.spatial.distance import cosine
# from sklearn.preprocessing import StandardScaler



In [ ]:
# TESTS:
# tested identical normalization
# tested identical standardization
# tested identical regression of (residual) AGE, SEX, and BATCH
# tested identiacal regression (residual) of AGE, SEX, BATCH, and CELL TYPE

In [46]:
# check arguments
# check that no columns or rows with all zeros
# ensure that factor variables are NOT integers or real
# what if no covariates
# subsetting to most variable genes, iterPCA and stand/cond PCA will be different
# output mean variance relationship
# ensure no rows or columns with 0 entries
#  MUST ADD VARGENES SUBSET HERE !

class condPCA(object):
    def __init__(self, count_matrix_path, metadata_path, object_columns, vars_to_regress=True, n_PCs=200, random_seed=9989999):
        """
        Parameters
        ----------
        count_matrix:
            Count matrix that must be QC'd

        metadata:
            metadata containing cell type labels named "celltype"

        object_columns:
            columns that will be one hot encoded/columns that are factors 

        vars_to_regress:
            list of variables to regress out

        """
        self.count_matrix = scanpy.read(count_matrix_path) # cells x genes, pd.read_csv(count_matrix_path, sep='\t', header=0, index_col=0)
       
        self.metadata = pd.read_csv(metadata_path, sep='\t', header=0, index_col=0)
        
        if vars_to_regress:
            
            self.vars_to_regress = self.metadata.columns
        
        else: # if vars_to_regress is a list, convert to pandas core Index object
            
            self.vars_to_regress = pd.Index(vars_to_regress)

        # one hot encode necessary metadata variables
        self.object_columns = object_columns # obtain columns that must be one hot encoded
        
        self.metadata[self.object_columns] = self.metadata[self.object_columns].astype(object) # convert these columns to objects

        self.random_seed = random_seed # set random seed
        
        self.n_PCs = n_PCs

    def Normalize(self):
        """ 
        Normalize and take log1p of count data
        """
        
        # update scanpy object to normalize all rows, so every cell sums to 10k
        scanpy.pp.normalize_total(self.count_matrix, target_sum = 10000) 
       
        # log transform
        scanpy.pp.log1p(self.count_matrix) 

    def Standardize(self):
        """ 
        Standardize count data AND metadata
        """
        
        # Standardize count data
        if scipy.sparse.issparse(self.count_matrix.X):
            
            self.count_matrix.X = self.count_matrix.X.todense()
        
        self.standardized_count_data = self._standardize(self.count_matrix.X)

        # Process metadata/covariates for standardization:

        # subset to only variables that you want to regress out
        self.metadata = self.metadata[self.vars_to_regress] 
       
        # WARNING IN FOLLOWING LINE BECAUSE CONVERTING OBJECT THAT LOOKS NUMERIC TO BE ONE HOT ENCODED, this is batch
        self.IterPCA_metadata = pd.get_dummies(self.metadata, drop_first=False)
        
        # Convert factor covariates to dummy variables dropping one column 
        self.metadata = pd.get_dummies(self.metadata, drop_first=True) 
        
        self.standardized_metadata = self._standardize(self.metadata)
    
    def _standardize(self, mat): # simple function performing standardization
        
        mean_vector = np.mean(mat, axis=0)
       
        std_vector = np.std(mat, axis=0)
        
        # standardize by gene
        stand_mat = (mat - mean_vector) / std_vector 
        return stand_mat
    
    def _regress_covariates(self, standardized_metadata, standardized_count_data): # function regressing set of covariates
        
        # append ones to standardized meta for intercept
        standardized_metadata = np.c_[np.ones((standardized_metadata.shape[0], 1)), standardized_metadata] 
        
        # compute inverse of np.matmul(A^T, A) where A is the standardized metadata or covariates
        inv_cov = np.linalg.pinv(np.matmul(standardized_metadata.T, standardized_metadata) ) 
        
        # compute betas per gene
        betas = np.apply_along_axis(self._column_wise_regression, axis=0, arr=standardized_count_data, inv_cov_mat=inv_cov, standardized_metadata_mat=standardized_metadata) 
        
        # compute prediction
        prediction = np.matmul(standardized_metadata, betas) # compute prediction
        
        # compute residual
        residual = standardized_count_data - prediction 
        
        standardized_residual = self._standardize(residual)
        
        return standardized_residual

    def _column_wise_regression(self, column, inv_cov_mat, standardized_metadata_mat): # perform regression of metadata/covariates across all genes
        
        # compute betas for a given gene (dimension of covariates plus 1 for intercept)
        betas = inv_cov_mat @ standardized_metadata_mat.T @ column 
        
        return betas
    
    def _fit_pca(self, mat): # fitting PCA
        
        # instantiate PCA with hyperparameters
        pca = PCA(n_components=self.n_PCs, random_state=self.random_seed) 
        
        # projections (of input data onto eigenvectors)
        pca.fit(mat) 
       
        # retrieve eigenvectors
        gene_loadings = pca.components_ 
        
        cell_embeddings = pca.transform(mat)
       
        eigenvalues = pca.explained_variance_ 
        
        return(cell_embeddings, gene_loadings, eigenvalues)
    
    def _compute_BIC(self, mat): # compute BIC significant PCs
        return 0
    
    def _fit_model(self, standardized_metadata, standardized_count_data, iterPCA=False, iterPCA_genenames=False, iterPCA_cellnames=False): # regress out covariates and then input into PCA
        #standardized_count_data= self.standardized_count_data
        # regress out covariates (including celltype) and retrieve standardized residual
        standardized_residual = self._regress_covariates(standardized_metadata = standardized_metadata, standardized_count_data= standardized_count_data)
        np.save('/Users/shayecarver/condPCA/final_method/standardized_residual.npy', standardized_residual)
        # if not iterative PCA, able to add gene names and cell names here, but must subset if IterPCA
        if not iterPCA: 
            # return standardized residual as a dataframe with gene and cell names:

            #  MUST ADD VARGENES SUBSET HERE !
            standardized_residual = pd.DataFrame(standardized_residual, index = list(self.count_matrix.obs_names), columns = list(self.count_matrix.var_names))

        if iterPCA:
            # return standardized residual as a dataframe with gene and cell names of the given subset:
            standardized_residual = pd.DataFrame(standardized_residual, index = list(iterPCA_cellnames), columns = list(iterPCA_genenames))

        # perform PCA on residualized matrix
        return( self._fit_pca(standardized_residual) )

    def _mapping_IterPCA_subset_dataframes_to_PCA(self, metadata, CT_column): 
        
        # extract indices of the cells that belong to the particular cell type of interest (indicated by CT_column, which is a column name)
        indices_given_ct = self.CT_indices_df[CT_column]
        
        # subset the count data to the cells belonging to the cell type
        count_data_subset_to_CT = self.standardized_count_data[indices_given_ct]
        
        # subset the metadata to the cells belonging to the cell type
        metadata_subset_to_CT = metadata[indices_given_ct]

        # Re-standardize count databecause it has just been subset
        count_data_subset_to_CT = self._standardize(count_data_subset_to_CT)
        # Re-standardize metadata because it has just been subset
        metadata_subset_to_CT = self._standardize(metadata_subset_to_CT)

        # extract the cell names or barcodes of the cells belonging to the cell type of interest
        cellnames = self.count_matrix.obs_names[indices_given_ct]

        # extract the gene names of the genes belonging to the most variable genes within that subset
        #  MUST ADD VARGENES SUBSET HERE !
        genenames = self.count_matrix.var_names

        # fit the given model by regressing out covariates and performing PCA
        return self._fit_model(standardized_metadata=metadata_subset_to_CT,standardized_count_data = count_data_subset_to_CT, iterPCA=True, iterPCA_genenames=genenames, iterPCA_cellnames = cellnames)

    def CondPCA_fit(self):
       
        # fit linear model (regress out covariates) and fit PCA -- covariates contain cell type
        self.COND_cell_embeddings, self.COND_gene_loadings, self.COND_eigenvalues = self._fit_model(standardized_metadata=self.standardized_metadata,standardized_count_data= self.standardized_count_data)

        # compute BIC

    def StandardPCA_fit(self):
        
        # remove celltype from covariate space
        metadata_minus_celltype = self.standardized_metadata.drop(columns = self.standardized_metadata.filter(like="celltype", axis=1).columns )
        
        # fit linear model (regress out covariates) and fit PCA -- covariates do not contain cell type
        self.STANDARD_cell_embeddings, self.STANDARD_gene_loadings, self.STANDARD_eigenvalues = self._fit_model(standardized_metadata=metadata_minus_celltype,standardized_count_data= self.standardized_count_data)
    
    def Iter_PCA_fit(self):

        # remove celltype from covariate space
        metadata_minus_celltype = self.standardized_metadata.drop(columns = self.standardized_metadata.filter(like="celltype", axis=1).columns )
        
        # get dataframe with boolean indices for each cell type
        self.CT_indices_df = self.IterPCA_metadata.filter(like="celltype", axis=1).astype(bool) 
        
        # get the name of the columns that indicate a cell type
        celltype_colnames = self.CT_indices_df.columns 

        # Create a partially applied function with the df to subset as the counts matrix
        subset_counts_matrix_fcn = partial(self._mapping_IterPCA_subset_dataframes_to_PCA, self.standardized_count_data)

        # output a list of PCA outputs
        self.result = list(map(subset_counts_matrix_fcn, celltype_colnames)) 

        # TO DO: MAKE SURE THAT THESE OUTPUTS ARE CORRECT?, perhaps make this variable a dictionary with cell type of interest

        # MUST RESTANDARDIZE
       

        



        

        


        
        

In [64]:
# instantiate class
test = condPCA(count_matrix_path="/Users/shayecarver/condPCA/final_method/test_matrix.txt", metadata_path="/Users/shayecarver/condPCA/final_method/test_metadata.txt", object_columns=['Batch', 'Sex','celltype']) # object_columns are columns that must be factors
test.Normalize()
test.Standardize()
test.CondPCA_fit()
#test.StandardPCA_fit()
#test.Iter_PCA_fit()

In [66]:
norm_a = test.count_matrix.X
scale_a = test.standardized_count_data
standardized_residual_batch_age_sex_A = np.load('/Users/shayecarver/condPCA/final_method/my_method_standardized_residual_test_matrix.npy')
standardized_residual_batch_age_sex_celltype_A = np.load('/Users/shayecarver/condPCA/final_method/standardized_residual.npy')

In [67]:
norm_b = np.array(pd.read_csv("/Users/shayecarver/condPCA/final_method/norm_test_matrix.txt", sep='\t', header=0, index_col=0))
scale_b = np.array(pd.read_csv("/Users/shayecarver/condPCA/final_method/scale_test_matrix.txt", sep='\t', header=0, index_col=0))
standardized_residual_batch_age_sex_B = np.array(pd.read_csv("/Users/shayecarver/condPCA/final_method/SEURAT_scale_counts_regress_batch_age_sex.txt", sep='\t', header=0, index_col=0)) # residual computed in seurat
standardized_residual_batch_age_sex_B = np.array(pd.read_csv("/Users/shayecarver/condPCA/final_method/base_R_standardized_residual_test_matrix.txt", sep='\t', header=0, index_col=0)) #residual computed in base R
standardized_residual_batch_age_sex_celltype_B = np.array(pd.read_csv("/Users/shayecarver/condPCA/final_method/base_R_standardized_residual_test_matrix_w_CELLTYPE.txt", sep='\t', header=0, index_col=0)) #residual computed in base R




In [62]:
def inner_prod_similarity(MatrixA, MatrixB): # compute inner product similarity between two matrices
    similarity = np.sum(MatrixA * MatrixB, axis=0) / (np.linalg.norm(MatrixA, axis=0) * np.linalg.norm(MatrixB, axis=0))
    print(f'sum should be ~2000: {sum(similarity)}')
    return similarity

In [68]:
inner_prod_similarity(norm_a,norm_b) 
inner_prod_similarity(scale_a,scale_b)
inner_prod_similarity(standardized_residual_batch_age_sex_A,standardized_residual_batch_age_sex_B)
inner_prod_similarity(standardized_residual_batch_age_sex_celltype_A,standardized_residual_batch_age_sex_celltype_B)

sum should be ~2000: 2000.0000395791126
sum should be ~2000: 2000.0001817499913
sum should be ~2000: 2000.0
sum should be ~2000: 1999.9999999999998


array([1., 1., 1., ..., 1., 1., 1.])